In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [3]:
def analyze_dividend_yield_complete(dividend_file, stock_file):
    
    # Load dividend data
    dividends = pd.read_csv(dividend_file)
    dividends = dividends.drop(columns=[dividends.columns[0]])  # Remove index column
    dividends['date'] = pd.to_datetime(dividends['date'])
    
    # Load stock data
    stocks = pd.read_csv(stock_file)
    stocks = stocks.drop(columns=[stocks.columns[0]])  # Remove index column
    stocks['date'] = pd.to_datetime(stocks['date'])
    stocks = stocks.rename(columns={'close_adjusted': 'price'})
    
    # Get 10-year date range
    end_date = max(stocks['date'].max(), dividends['date'].max())
    start_date = end_date - timedelta(days=10*365)
    
    # Filter to 10-year period
    dividends_10y = dividends[dividends['date'] >= start_date].copy()
    stocks_10y = stocks[stocks['date'] >= start_date].copy()
    
    # Calculate current dividend yield for each symbol
    latest_prices = stocks_10y.groupby('symbol')['price'].last().reset_index()
    recent_dividends = dividends_10y[dividends_10y['date'] >= (end_date - timedelta(days=365))]
    annual_dividends = recent_dividends.groupby('symbol')['dividend'].sum().reset_index()
    
    # Merge and calculate yield
    current_yield = pd.merge(annual_dividends, latest_prices, on='symbol', how='inner')
    current_yield['yield_pct'] = (current_yield['dividend'] / current_yield['price']) * 100
    
    # Get top 10 symbols
    top_10_symbols = current_yield.nlargest(10, 'yield_pct')['symbol'].tolist()
    
    # Calculate rolling 12-month dividend yield
    results = []
    for symbol in top_10_symbols:
        symbol_stocks = stocks_10y[stocks_10y['symbol'] == symbol].copy()
        symbol_divs = dividends_10y[dividends_10y['symbol'] == symbol].copy()
        
        # Monthly resampling for efficiency
        symbol_stocks['year_month'] = symbol_stocks['date'].dt.to_period('M')
        monthly_prices = symbol_stocks.groupby('year_month')['price'].last().reset_index()
        monthly_prices['date'] = monthly_prices['year_month'].dt.to_timestamp()
        
        for _, row in monthly_prices.iterrows():
            current_date = row['date']
            current_price = row['price']
            
            # Get 12-month trailing dividends
            twelve_months_ago = current_date - timedelta(days=365)
            trailing_divs = symbol_divs[
                (symbol_divs['date'] >= twelve_months_ago) & 
                (symbol_divs['date'] <= current_date)
            ]
            
            if len(trailing_divs) > 0:
                total_dividends = trailing_divs['dividend'].sum()
                yield_pct = (total_dividends / current_price) * 100
                
                results.append({
                    'symbol': symbol,
                    'date': current_date,
                    'dividend_yield': yield_pct
                })
    
    return pd.DataFrame(results), current_yield

In [4]:
def create_dividend_yield_plot(df_results, title_suffix=""):
    """Create plotly visualization of dividend yield data."""
    
    fig = go.Figure()
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    
    symbols = df_results['symbol'].unique()
    
    for i, symbol in enumerate(symbols):
        symbol_data = df_results[df_results['symbol'] == symbol]
        
        fig.add_trace(go.Scatter(
            x=symbol_data['date'],
            y=symbol_data['dividend_yield'],
            mode='lines+markers',
            name=symbol,
            line=dict(color=colors[i % len(colors)], width=2),
            marker=dict(size=4),
            hovertemplate=f'<b>{symbol}</b><br>' +
                        'Date: %{x}<br>' +
                        'Dividend Yield: %{y:.2f}%<br>' +
                        '<extra></extra>'
        ))
    
    fig.update_layout(
        title={
            'text': f'Rolling 12-Month Dividend Yield Analysis{title_suffix}',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16}
        },
        xaxis_title='Date',
        yaxis_title='Dividend Yield (%)',
        hovermode='x unified',
        legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02),
        width=1200,
        height=600,
        template='plotly_white'
    )
    
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
    
    return fig

In [5]:
def get_top_dividend_yields(dividend_file, n=10):
    """
    Get top N companies by recent dividend activity (when stock data unavailable).
    """
    dividends = pd.read_csv(dividend_file)
    dividends = dividends.drop(columns=[dividends.columns[0]])
    dividends['date'] = pd.to_datetime(dividends['date'])
    
    # Get recent dividends (last 2 years)
    recent_cutoff = dividends['date'].max() - timedelta(days=730)
    recent_dividends = dividends[dividends['date'] >= recent_cutoff]
    
    # Calculate total dividends per symbol
    total_dividends = recent_dividends.groupby('symbol')['dividend'].sum().reset_index()
    top_symbols = total_dividends.nlargest(n, 'dividend')
    
    print(f"Top {n} companies by total dividend payments (last 2 years):")
    for _, row in top_symbols.iterrows():
        print(f"{row['symbol']}: ${row['dividend']:.2f}")
    
    return top_symbols['symbol'].tolist()

In [6]:
def run_analysis(dividend_file='sp500_historical_dividends.csv', stock_file='sp500_historical_daily_close.csv'):
    
    print("=== S&P 500 Dividend Yield Analysis ===\n")
    
    # Run the complete analysis
    df_results, current_yields = analyze_dividend_yield_complete(dividend_file, stock_file)
    
    if len(df_results) > 0:
        print(f"\nGenerated {len(df_results)} data points for visualization")
        print(f"Top 10 companies by current dividend yield:")
        print(current_yields.nlargest(10, 'yield_pct')[['symbol', 'yield_pct']])
        
        fig = create_dividend_yield_plot(df_results, " - Top 10 S&P 500 Companies")
        
        # Show summary statistics
        print("\nSummary Statistics:")
        for symbol in df_results['symbol'].unique():
            symbol_data = df_results[df_results['symbol'] == symbol]
            avg_yield = symbol_data['dividend_yield'].mean()
            max_yield = symbol_data['dividend_yield'].max()
            min_yield = symbol_data['dividend_yield'].min()
            print(f"{symbol}: Avg={avg_yield:.2f}%, Max={max_yield:.2f}%, Min={min_yield:.2f}%")
        
        return fig, df_results
    else:
        print("No overlapping data found. Showing top dividend companies by total payments:")
        top_symbols = get_top_dividend_yields(dividend_file, 10)
        return None, None


In [8]:
fig, data = run_analysis()

=== S&P 500 Dividend Yield Analysis ===


Generated 1178 data points for visualization
Top 10 companies by current dividend yield:
    symbol   yield_pct
16     AIV  120.083682
86     CTL    7.164402
309      T    7.152682
246    OKE    7.083333
295    SLG    6.566708
188    KMI    5.713128
351    XOM    5.710535
219     MO    5.226904
259    PGR    5.083515
90     CVX    5.057761

Summary Statistics:
AIV: Avg=12.68%, Max=205.45%, Min=0.52%
CTL: Avg=9.91%, Max=18.21%, Min=3.11%
T: Avg=5.99%, Max=7.66%, Min=2.04%
OKE: Avg=7.90%, Max=30.03%, Min=2.12%
SLG: Avg=2.95%, Max=8.40%, Min=0.13%
KMI: Avg=4.66%, Max=13.43%, Min=1.44%
XOM: Avg=4.02%, Max=10.80%, Min=0.70%
MO: Avg=5.29%, Max=9.37%, Min=1.86%
PGR: Avg=3.22%, Max=7.43%, Min=1.13%
CVX: Avg=4.13%, Max=7.35%, Min=0.99%


In [9]:
fig.show()